In [2]:
from typing import Callable, Dict, Iterable, Self
from numpy.typing import NDArray
import datasets
import tensorflow as tf
import numpy as np
import einops
import pandas as pd
import pickle

# import flagon

In [7]:
with open("data/solar_home_data.pkl", 'rb') as f:
    data = pickle.load(f)

def get_customer_data(customer=1):
    idx = np.arange(24, len(data[customer]))
    expanded_idx = np.array([np.arange(i - 24, i - 1) for i in idx])
    return data[customer][expanded_idx].reshape(len(data[customer]) - 24, -1), data[customer][idx, 0]

In [10]:
get_customer_data(1)[0].shape

(8736, 92)

In [82]:
def create_model() -> tf.keras.Model:
    inputs = tf.keras.Input((92))
    # x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Reshape((23, 4))(inputs)
    x = tf.keras.layers.Conv1D(32, 5, activation="relu")(x)
    x = tf.keras.layers.Conv1D(32, 3, activation="relu")(x)
    x = tf.keras.layers.Conv1D(32, 2, activation="relu")(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(50, activation="relu")(x)
    x = tf.keras.layers.Dense(1, activation="linear")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"]
    )
    return model

In [13]:
X, Y = get_customer_data(1)

In [83]:
model = create_model()
model.fit(X, Y, epochs=10)

Epoch 1/10
273/273 [==============================] - 1s 904us/step - loss: 9.6492 - mean_absolute_error: 1.0454
Epoch 2/10
273/273 [==============================] - 0s 906us/step - loss: 0.8027 - mean_absolute_error: 0.6814
Epoch 3/10
273/273 [==============================] - 0s 932us/step - loss: 0.6685 - mean_absolute_error: 0.6176
Epoch 4/10
273/273 [==============================] - 0s 908us/step - loss: 0.5966 - mean_absolute_error: 0.5839
Epoch 5/10
273/273 [==============================] - 0s 915us/step - loss: 0.5688 - mean_absolute_error: 0.5694
Epoch 6/10
273/273 [==============================] - 0s 911us/step - loss: 0.5719 - mean_absolute_error: 0.5692
Epoch 7/10
273/273 [==============================] - 0s 912us/step - loss: 0.5415 - mean_absolute_error: 0.5566
Epoch 8/10
273/273 [==============================] - 0s 918us/step - loss: 0.5319 - mean_absolute_error: 0.5465
Epoch 9/10
273/273 [==============================] - 0s 913us/step - loss: 0.5207 - mean_absolu

In [79]:
model.predict(X[Y < 0])

86/86 [==============================] - 0s 582us/step


array([[ 0.34779292],
       [ 0.99771875],
       [-0.21192604],
       ...,
       [-0.48255605],
       [-0.38452524],
       [ 0.669403  ]], dtype=float32)